This notebook compares the generation capabilities of an RNN to a Markov chain method.

The RNN code comes from TensorFlow, while the Markov chain is self-written.

# Text generation with an RNN (Taken from TensorFlow website)

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

### Read the data

First, look in the text:

In [2]:
# Read, then decode for py2 compat.
text = open('coding.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 22018 characters


In [3]:
''.join(sorted(list(set(text))))

'\n !"#%\'()*+,-./0123456789:;<=>@ABCDEFGHIKLMNOPQRSTUVZ[\\]_`abcdefghijklmnopqrstuvwxyz{|}'

In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

# Python3 program to add two numbers
  
number1 = input("First number: ")
number2 = input("\nSecond number: ")
  
# Adding two numbers
# User might also enter float numbers
sum = float(number1) + float(number2)
  
# Display the sum
# will print value


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

87 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(22018,), dtype=int64, numpy=array([ 5,  2, 46, ...,  9,  1,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(30):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

#
 
P
y
t
h
o
n
3
 
p
r
o
g
r
a
m
 
t
o
 
a
d
d
 
t
w
o
 
n


In [16]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'#' b' ' b'P' b'y' b't' b'h' b'o' b'n' b'3' b' ' b'p' b'r' b'o' b'g'
 b'r' b'a' b'm' b' ' b't' b'o' b' ' b'a' b'd' b'd' b' ' b't' b'w' b'o'
 b' ' b'n' b'u' b'm' b'b' b'e' b'r' b's' b'\n' b' ' b' ' b'\n' b'n' b'u'
 b'm' b'b' b'e' b'r' b'1' b' ' b'=' b' ' b'i' b'n' b'p' b'u' b't' b'('
 b'"' b'F' b'i' b'r' b's' b't' b' ' b'n' b'u' b'm' b'b' b'e' b'r' b':'
 b' ' b'"' b')' b'\n' b'n' b'u' b'm' b'b' b'e' b'r' b'2' b' ' b'=' b' '
 b'i' b'n' b'p' b'u' b't' b'(' b'"' b'\\' b'n' b'S' b'e' b'c' b'o' b'n'
 b'd' b' ' b'n'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())
  print(len(seq))

b'# Python3 program to add two numbers\n  \nnumber1 = input("First number: ")\nnumber2 = input("\\nSecond n'
101
b'umber: ")\n  \n# Adding two numbers\n# User might also enter float numbers\nsum = float(number1) + float('
101
b'number2)\n  \n# Display the sum\n# will print value in float\nprint("The sum of {0} and {1} is {2}" .form'
101
b'at(number1, number2, sum))\n\n# Python 3 program to find\n# factorial of given number\ndef factorial(n):\n'
101
b'     \n    # single line to find factorial\n    return 1 if (n==1 or n==0) else n * factorial(n - 1);\n '
101


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'# Python3 program to add two numbers\n  \nnumber1 = input("First number: ")\nnumber2 = input("\\nSecond '
Target: b' Python3 program to add two numbers\n  \nnumber1 = input("First number: ")\nnumber2 = input("\\nSecond n'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [24]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru1.get_initial_state(x)
    x, states = self.gru1(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 88) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  22528     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  90200     
                                                                 
Total params: 4,051,032
Trainable params: 4,051,032
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [29]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 88), dtype=float32, numpy=
array([[ 0.00619111,  0.01501912,  0.0084628 , ...,  0.00469326,
         0.00330961,  0.00556275],
       [ 0.0028591 ,  0.01420797,  0.01434019, ..., -0.0018965 ,
         0.00103801,  0.00642334],
       [ 0.00432305,  0.00290702,  0.01420512, ..., -0.00889909,
         0.00498802, -0.00583102],
       ...,
       [ 0.00538473, -0.00366278,  0.01225198, ..., -0.00550904,
        -0.00380149, -0.00301143],
       [ 0.00082919,  0.00264745,  0.01660097, ..., -0.00432567,
         0.00227055,  0.01271428],
       [-0.00127503, -0.00074118,  0.0017617 , ..., -0.00148645,
        -0.00092464,  0.01950732]], dtype=float32)>

In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [31]:
sampled_indices

array([40, 38, 41,  4, 58, 76, 81, 87,  5, 46, 30, 39, 55, 12, 24, 11, 79,
       59, 56, 33, 62, 58, 41, 40, 24, 54,  7, 13, 86, 15,  4, 52, 82, 21,
       39, 76, 14, 53, 14, 46, 82, 45, 48, 52, 29, 36, 42,  2, 64, 54,  4,
        0, 77, 34, 68, 15, 65, 26, 31, 76, 31, 67, 37, 74, 43, 53, 61, 84,
       51, 50, 49, 45, 14, 60, 79, 10,  3, 23, 51, 43, 18, 83, 85, 57, 48,
       67, 49, 81, 32, 66,  5, 29, 27, 76, 76, 43,  9, 21, 33,  4])

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' tensor. (N,)\n    ys: tuple of\n        decoder_input: int32 tensor. (N, T2)\n        y: int32 tensor.'

Next Char Predictions:
 b'IGK"`rw}#P>H\\,8+ua]Bd`KI8[\'-|/"Vx5Hr.Z.PxORV=EL f["[UNK]sCj/g:@r@iFpMZczUTSO.bu*!7UM2y{_RiSwAh#=;rrM)5B"'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 88)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4730744, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

87.62571

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [36]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [38]:
EPOCHS = 100

In [55]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
3/3 [==============================] - 1s 231ms/step - loss: 0.0500
Epoch 2/100
3/3 [==============================] - 1s 213ms/step - loss: 0.0481
Epoch 3/100
3/3 [==============================] - 1s 198ms/step - loss: 0.0468
Epoch 4/100
3/3 [==============================] - 1s 185ms/step - loss: 0.0469
Epoch 5/100
3/3 [==============================] - 1s 185ms/step - loss: 0.0479
Epoch 6/100
3/3 [==============================] - 1s 196ms/step - loss: 0.0455
Epoch 7/100
3/3 [==============================] - 1s 180ms/step - loss: 0.0448
Epoch 8/100
3/3 [==============================] - 1s 179ms/step - loss: 0.0436
Epoch 9/100
3/3 [==============================] - 1s 194ms/step - loss: 0.0434
Epoch 10/100
3/3 [==============================] - 1s 183ms/step - loss: 0.0432
Epoch 11/100
3/3 [==============================] - 1s 191ms/step - loss: 0.0420
Epoch 12/100
3/3 [==============================] - 1s 178ms/step - loss: 0.0425
Epoch 13/100
3/3 [===================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [52]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [53]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text.

In [60]:
start = time.time()
states = None
next_char = tf.constant(['import tensorflow'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

import tensorflow as tf

def ln(inputs, epsilon=0.1):
    '''Applies label smoothing. See 5.4 and https://arxiv.org/abs/1607.00000000)
           ensidin=True):
        '''
        with tf.variable_scope("encoder", reuse=tf.AUTO_REUSE):
            decoder_inputs, y, seqlens
    3d tensor that has the same shape as inputs.
    '''

    E = inputs.get_shape().as_list()[-1] # number of channels
    return ((1-epsilon) * inputs)
        training: boolean. If true, units that referencert
    outputs /= gmabedang.
        outputs = tf.where(tf.equal(future_masks, 0), paddings, inputs)
    else:
            ep = tf.ratrog.
(Noze)        lrosting in ("be, cousalis):
        outputs = inputs*gatse
fiodtr tianal scooterong

            outputs = tf.where(tf.equal(future_masks, 0), paddings, inputs)
    else:
            em =mamsks e float numbers
        
        # Normalize
        outputs = ln(outputs)
    
    return tf.to_float(outputs)

def noam_scheme
initral in'
ritg.
    dropout_rate: A

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO:,\n ent isitr = nteples: (valuy) / ( -w.ing: atf.hp.mask(oE", uses2is)\n\n          c[ = ys\n    (   [ Adp, decoser_hat),\n                                   = gloat(pos=tes, sents1, srm_masksis=N).er\n# train_loge( *  encode(self.hp.dropout_rate, yshed ypser"forwwiwi\n\'\'\n        mead = tf.concat(tf.dx, dem_hest_s, self.haten)\n\n      tro tinl pars. d fore_ant rets.rensor (N, T, E)\n    \'\'\'\npre tf.innctimizer(gutm_sist() padding_baihslopanloby, shape f renurit weights  the sarm byeusepose\n in tensor. (N,)\n        # flrat print pare in True fiod lavelr.\nbe  hpapads.1: s, stype = (inputs)\n        sents2: (N, T1)\n    \n      pes: At2.\nxlas /am shmpe as paxdrimsurits, utiontive)\n\n        he masking: (N,,\n        self.eqdens1(= (hp.tosint()[-\n    return (N, T2, d_model)\n\n      \n         lostinn = inputs.1od\n# "utivey Plint amporo self.embeddin.self, ouper2")\n\n      # ef colutt.\n    \n    position_encolins: (N, T_q, E)\n\n    \'\'\'mienod at aqdil

## Markov Chain Method

Do a 1 letter prediction based on the previous n characters

- **** a b c d e f g h i j k . . .
- [aa] 0 0 0 0 0 0 0 0 0 0 0 0 0 0
- [ab] 1 0 0 0 0 0 0 0 0 0 0 0 0 0
- [ac]

Count the occurences of the next character in sets of (n+1) characters, and update the frequency array

When sampling, just sample the next character based on a lookup table of the first n characters.

In [44]:
len(text)
uniq = list(set(text))
print(uniq)

['b', 'j', '<', 'Q', 'E', 'e', 'Z', 'o', ')', '5', 'k', '_', '\n', 'g', ' ', 'v', '"', 'C', 'c', 'B', 'N', ':', '.', '{', 'y', '!', 'F', "'", '}', 'l', 't', '+', '\\', 'V', '-', '=', ',', 'A', ';', '[', 'H', ']', 'U', 'r', 'i', '@', '`', 'M', '(', '1', 'T', 'D', '/', '9', '*', 'a', 'P', 'x', '3', 'u', '0', '4', 'G', '%', '7', 'f', 'p', 'z', 'n', 's', 'K', 'I', 'd', 'h', 'w', '2', 'R', 'q', '6', '8', '#', 'S', '>', 'L', 'm', 'O', '|']


In [45]:
b = {x:0 for x in uniq}
b['x'] += 1
print(b)

{'b': 0, 'j': 0, '<': 0, 'Q': 0, 'E': 0, 'e': 0, 'Z': 0, 'o': 0, ')': 0, '5': 0, 'k': 0, '_': 0, '\n': 0, 'g': 0, ' ': 0, 'v': 0, '"': 0, 'C': 0, 'c': 0, 'B': 0, 'N': 0, ':': 0, '.': 0, '{': 0, 'y': 0, '!': 0, 'F': 0, "'": 0, '}': 0, 'l': 0, 't': 0, '+': 0, '\\': 0, 'V': 0, '-': 0, '=': 0, ',': 0, 'A': 0, ';': 0, '[': 0, 'H': 0, ']': 0, 'U': 0, 'r': 0, 'i': 0, '@': 0, '`': 0, 'M': 0, '(': 0, '1': 0, 'T': 0, 'D': 0, '/': 0, '9': 0, '*': 0, 'a': 0, 'P': 0, 'x': 1, '3': 0, 'u': 0, '0': 0, '4': 0, 'G': 0, '%': 0, '7': 0, 'f': 0, 'p': 0, 'z': 0, 'n': 0, 's': 0, 'K': 0, 'I': 0, 'd': 0, 'h': 0, 'w': 0, '2': 0, 'R': 0, 'q': 0, '6': 0, '8': 0, '#': 0, 'S': 0, '>': 0, 'L': 0, 'm': 0, 'O': 0, '|': 0}


In [46]:
import numpy as np

def sample(input_dict):
  value_counts = []
  char_array = []
  for key,value in input_dict.items():
    char_array.append(key)
    value_counts.append(value)

  value_counts = np.array(value_counts)
  return np.random.choice(char_array, p=list(value_counts/sum(value_counts)))[0]

In [47]:
n=1
onechar = {}
for i in range(len(text)-n-1):
  reference, next_value = text[i:i+n], text[i+n]
  # if it is in dictionary, add one to the next character
  if reference in onechar:
    onechar[reference][next_value] += 1
  # otherwise, give it the initial values
  else:
    onechar[reference] = {x:0 for x in uniq}
    onechar[reference][next_value] += 1

print(f'Number of unique {n} character sequences: {len(onechar)}')

Number of unique 1 character sequences: 87


In [67]:
# Replace the value of n here for the last n characters
n=7
d = {}
for i in range(len(text)-n-1):
  reference, next_value = text[i:i+n], text[i+n]
  # if it is in dictionary, add one to the next character
  if reference in d:
    d[reference][next_value] += 1
  # otherwise, give it the initial values
  else:
    d[reference] = {x:0 for x in uniq}
    d[reference][next_value] += 1

print(f'Number of unique {n} character sequences: {len(d)}')

Number of unique 7 character sequences: 10167


In [65]:
# The distribution of characters given a starting n characters
# print(d['ROMEO'])

Now we can then sample and update the next 1000 characters

In [77]:
myinput = "import tensorflow"
for i in range(1000):
  # if there is something seen before, go by the probability distribution
  if myinput[-n:] in d:
    myinput += sample(d[myinput[-n:]])
  # if not, then just random sample based on just one character
  else:
    myinput += sample(onechar[myinput[-1]])

print(myinput)


import tensorflow as tf

from data_load import convert_to_tensor
from tqdm import logging.info("# write results")
    hypotheses(num_test_batch(hp.testdir): os.makedirs(hp.test_batches, num_units: embedding
                     scope: Optional scope for `variable_scope(scope, reuse=tf.AUTO_REUSE):
    '''Predicts autoregressively
                           tf.summary.text("pred", pred)
            values of the first row (id = 0) should be constant zero
    To apply the cosine to find
# factorial of given principal amount, time and
# rate of interest(8, 6, 8)

# -*- coding: utf-8 -*-
#/usr/bin/python2
'''
Feb. 2019 by kyubyong/transformer network
'''
import numpy as np
import convert_to_tensor([[[0, 0, 1], 
       summary.scalar("global_step()
                if type in ("f", "future", "right"):
         # Vanilla attention(queries. 3d tensor with shape of [N, key_seqlen]
    causality:
                      outputs /= d_k ** 0.5  # scale
         Returns
         n = tf.to_float(tf.no